In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy
import keras
from keras import backend as K

from keras.layers.merge import Concatenate
from keras.layers import Lambda
from keras.layers.core import Reshape

import numpy as np

# 1. Mixture of Gaussian Output

In [7]:
def mixture_of_gaussian_output(x, n_components):
    mu      = keras.layers.Dense(n_components, activation='linear')(x)
    log_sig = keras.layers.Dense(n_components, activation='linear')(x)
    pi      = keras.layers.Dense(n_components, activation='softmax')(x)
    return Concatenate(axis=1)([pi, mu, log_sig])

In [8]:
from keras.layers import Input, Dense
from keras.models import Model

In [10]:
n_components = 10
GM_coef_input = Input(shape = (32,), name = 'GM_coef_input')

GM_coef_est   = mixture_of_gaussian_output(GM_coef_input, n_components)

GM_coef_model = Model(inputs = GM_coef_input, 
                      output = GM_coef_est)

GM_coef_model

/Users/floyd/Environments/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("GM..., outputs=Tensor("co...)`
  import sys


In [12]:
GM_coef_model.compile('adam', loss='mse')

In [35]:
x = np.random.randn(32)
x = x.reshape(1,32)
x.shape

(1, 32)

In [36]:
x  # 32

array([[ 0.13956996,  0.106548  , -1.63864014, -0.21124345, -1.33505464,
        -1.67440698, -1.64551394,  0.46850645,  0.93785556,  0.05488123,
         0.4662185 , -1.6386602 , -0.52623273, -1.50079625, -1.93973896,
        -0.57059798,  1.26395749, -0.85389492, -1.21701524,  1.28780954,
        -1.18867975,  0.31669076, -0.3802014 , -1.17859141, -0.16894064,
         0.23787471, -1.11929253,  1.48657053, -0.89567899,  0.82242094,
         0.81376111, -0.85449572]])

In [39]:
est = GM_coef_model.predict(x) # 30
est

array([[ 0.17760766,  0.00609762,  0.07993325,  0.01104008,  0.0523461 ,
         0.24972679,  0.3087008 ,  0.02423287,  0.03855328,  0.05176151,
        -2.26054192, -0.21945223, -0.45609069,  0.89338118, -0.93940842,
        -0.60538924, -1.76229751,  0.76731324,  0.36846834,  1.79476452,
        -0.70900208, -2.15119624, -0.3035385 , -0.19439793,  0.27171034,
        -0.52836764, -0.71464032,  1.775406  , -0.72344625, -1.26091707]], dtype=float32)

In [40]:
GM_coef_est

<tf.Tensor 'concatenate_2/concat:0' shape=(?, 30) dtype=float32>

# 2. Mixture of Gaussian Loss

In [47]:
def split(start, stop):
    return Lambda(lambda x: x[:, start:stop], output_shape=(None, stop-start))

In [48]:
# here x is the GM_coef_est
def split_mixture_of_gaussians(x, n_components):
    pi = split(0, n_components)(x)
    mu = split(n_components, 2*n_components)(x)
    log_sig = split(2*n_components, 3*n_components)(x)
    return pi, mu, log_sig

In [49]:
# 逐元素clip（将超出指定范围的数强制变为边界值）
# K.clip?

In [65]:
def log_norm_pdf(x, mu, log_sig, log = True):
    z = (x - mu) / (K.exp(K.clip(log_sig, -40, 40))) #TODO: get rid of this clipping

    return -(0.5)*K.log(2*numpy.pi) - log_sig - 0.5*((z)**2)

In [55]:
y_pred = GM_coef_est
pi, mu, log_sig = split_mixture_of_gaussians(y_pred, n_components)

print(pi)
print(mu)
print(log_sig)

Tensor("lambda_7/strided_slice:0", shape=(?, 10), dtype=float32)
Tensor("lambda_8/strided_slice:0", shape=(?, 10), dtype=float32)
Tensor("lambda_9/strided_slice:0", shape=(?, 10), dtype=float32)


In [ ]:
x.set_shape([None, 1])

In [5]:
def mix_gaussian_loss(price_true, mu, log_sig, pi):
    '''
    Combine the mixture of gaussian distribution and 
    the loss into a single function
    so that we can do the log sum exp trick for numerical stability...
    '''
    if K.backend() == "tensorflow":
        price_true.set_shape([None, 1])
    log_norm_value = log_norm_pdf(K.repeat_elements(x=price_true, rep=mu.shape[1], axis=1), 
                                  mu, 
                                  log_sig)
    # TODO: get rid of clipping.
    log_norm_value = K.clip(log_norm_value, -40, 40)
    # K.maximum 逐元素取两个张量的最大元素
    log_norm_value_max = K.maximum((0.), K.max(log_norm_value))
    # if log_norm_value_max is 0
    #     which means **norm_value** are all samller than 1
    # if log_norm_value_max is bigger than 1
    #     then, log_norm_value = log_norm_value - log_norm_value_max
    #     makes every element in **norm_value** is samller than 1
    # log sum exp trick...
    log_norm_value = log_norm_value - log_norm_value_max
    out = K.sum(pi * K.exp(log_norm_value), axis=1)
    # -K.log(out) is the key.
    loss = K.mean(-K.log(out) + log_norm_value_max) 
    return loss

In [64]:
import scipy as sp

from scipy.stats import norm

norm.pdf(np.random.randn(10))

array([ 0.24751594,  0.39893763,  0.37075068,  0.0854133 ,  0.33643236,
        0.35966641,  0.37529869,  0.30508083,  0.03901266,  0.39857178])

In [7]:
def mixture_of_gaussian_loss(price_true, GM_coef_est, n_components):
    pi, mu, log_sig = split_mixture_of_gaussians(GM_coef_est, n_components)
    return mix_gaussian_loss(price_true, mu, log_sig, pi)

# 3. Summary

In [8]:
def mixture_gaussian(n_components):
    '''
    Build a mixture of gaussian output and loss function that may be used in a keras graph.
    '''

    def output(x):
        return mixture_of_gaussian_output(x, n_components)

    def keras_loss(y, x):
        return mixture_of_gaussian_loss(y, x, n_components)
    return output, keras_loss

